In [ ]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=a46d3bc5b20e5b0ec5ec396b9ea9a43f9519d936311d7ce986cbafb8c5d78366
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60943 sha256=effaa85d844295151abf47ac31214ff2d061e41c292800a50a839a202fc0f80f
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f53

In [ ]:
import torch
import segmentation_models_pytorch as smp
from moviepy.editor import ImageSequenceClip
import glob
import cv2
import numpy as np
import os

In [ ]:
PATH = '/content/drive/MyDrive/chip-segmentation/models/20231031-1.pth'

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda:0')
  model = torch.load(PATH)
else:
  device = torch.device('cpu')
  model = torch.load(PATH,map_location=torch.device('cpu'))
model = model.to(device)
model

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [ ]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'

In [ ]:
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
gif = cv2.VideoCapture('/content/drive/MyDrive/chip-segmentation/original-data/1.gif')
fps = gif.get(cv2.CAP_PROP_FPS)

In [ ]:
images = []
predicts = []
i = 0
while True:
    is_success, image_src = gif.read()
    if not is_success:
        break

    image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    # 前処理
    image = preprocessing_fn(image_src)
    image = image.transpose(2, 0, 1).astype('float32')
    # モデルで推論
    image=torch.from_numpy(image).to(device).unsqueeze(0)
    predict = model(image)
    predict = predict.detach().cpu().numpy()[0].reshape((256,256))

    # 0.5以上を1とする
    predict_img = np.zeros([256,256]).astype(np.int8)
    predict_img = np.where(predict>0.5, 255 , predict_img)
    images.append(image_src)
    predicts.append(predict_img)
    i += 1

In [ ]:
len(predicts),len(images)

(53, 53)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image


ims = []

for i in range(len(images)):
  # 横並び
  fig = plt.figure(figsize=(12, 6))
  ax1 = fig.add_subplot(1, 2, 1)
  ax2 = fig.add_subplot(1, 2, 2)
  ax1.imshow(images[i])
  ax2.imshow(predicts[i])

  fig.canvas.draw()

  data = fig.canvas.tostring_rgb()

  # 画像の大きさを取得する。
  w, h = fig.canvas.get_width_height()
  c = len(data) // (w * h)

  # PIL.Image に変換する
  img = Image.frombytes("RGB", (w, h), data, "raw")
  ims.append(img)

  plt.close()


ims[0].save('/content/drive/MyDrive/chip-segmentation/original-data/1-2.gif',
               save_all=True, append_images=ims[1:], optimize=False, duration=40, loop=0)